## **[주소 데이터 분석 + 지오맵]**

### **지리 정보 분석 후 맵 생성하기**

#### **0. 분석 미리보기**
---

* 목표: 특정 주소에 대한 지리 정보를 분석한 후, 위치를 시각화한 맵을 생성한다.
* 핵심 개념: 위도와 경도의 GPS 좌표, 지리 정보 분석, 지리 정보 시각화 라이브러리
* 데이터 수집:
    1. 커피 매장의 주소 데이터
    2. 행정구역 주소체계 데이터
* 데이터 준비 및 탐색:
    1. 데이터 정제: 주소의 행정구역 이름을 정확한 이름으로 수정
    2. 데이터 조합: 필요한 컬럼을 추출하고 병합

#### **1. 핵심 개념 이해**
---

프로젝트를 수행하기 위해 필요한 내용을 살펴보자.

* GPS 좌표 구하기: 특정 위치의 GPS 좌표를 나타내는 지오 데이터는 구글맵이나 네이버 지도 서비스에서 구할 수 있다. 또는 오픈 소프트웨어인 Geocoder-Xr을 사용해 구할 수도 있다.
* 지리 정보 시각화 라이브러리: 지오 데이터를 지도에 시각화하려면 지리 정보 시각화 라이브러리를 사용한다. 이때 포리움(folium)을 사용한다.

#### **분석할 커피 매장의 주소 데이터 준비하기**
---

In [1]:
import pandas as pd

In [3]:
CB = pd.read_csv(
    './CoffeeBean.csv',
    encoding='CP949',
    index_col=0,
    header=0,
    engine='python',
)

In [4]:
CB.head()

,store,address,phone
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052
3,신사점,서울시 강남구 도산대로 126,02-548-2741
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051


#### **주소 데이터를 행정구역 주소 체계에 맞게 정리하기**
---

'행정구역_시군구_별__성별_인구수_2.xlsx' 파일에 있는 정확한 시군구 이름과 비교하면서 CB 객체의 address 컬럼 값을 수정한다.
address 컬럼 값을 띄어쓰기 기준으로 분리한 후에 시도 이름을 변경하면 된다.

In [5]:
addr = []
for address in CB.address:
    addr.append(str(address).split())

In [6]:
addr

[['서울시', '강남구', '논현로', '566', '강남차병원1층'],
 ['서울시', '서초구', '강남대로', '369', '1층'],
 ['서울시', '강남구', '압구정로', '461', '네이처포엠빌딩B108,109호'],
 ['서울시', '강남구', '도산대로', '126'],
 ['서울시', '강남구', '논현로', '512', '지상1,2층'],
 ['서울시', '서초구', '강남대로', '213', '24호', '지하1층'],
 ['서울시', '강남구', '삼성로', '716', 'LEE76빌딩2층'],
 ['서울', '서초구', '반포동', '736-17', 'P빌딩', '2층'],
 ['서울시', '강남구', '언주로', '30길', '10,112', '현대비젼21', '112호'],
 ['서울시', '강남구', '선릉로', '749', '1,2층'],
 ['서울시', '강남구', '도산대로49길', '13', '1층', '17,18호'],
 ['서울시', '강남구', '선릉로', '157길', '12', '석전빌딩', '1~4층'],
 ['서울시', '서초구', '강남대로', '51길', '1', '511', 'TOWER', '1층'],
 ['서울시', '강남구', '논현', '231-13호', '팍스타워지하1층'],
 ['서울', '강남구', '테헤란로87길', '46', '지하', '2층'],
 ['서울시', '강남구', '영동대로', '511', '트레이드타워', '지하1층'],
 ['서울시', '강남구', '영동대로', '607', '1,2층'],
 ['서울시', '송파구', '석촌호수로', '118', '1층'],
 ['서울시', '서초구', '서초동', '1685-8호', '101~2호,113~4호,121호'],
 ['서울시', '강남구', '논현로', '717', '1층'],
 ['서울시', '서초구', '서초대로74길', '11', '지하2층'],
 ['서울시', '서초구', '서초대로74길', '4', '삼성생명보험서초

In [10]:
addr2 = []

for i in range(len(addr)):
    if addr[i][0] == '서울': addr[i][0] = '서울특별시'
    elif addr[i][0] == '서울시': addr[i][0] = '서울특별시'
    elif addr[i][0] == '부산시': addr[i][0] = '부산광역시'
    elif addr[i][0] == '인천': addr[i][0] = '인천광역시'
    elif addr[i][0] == '광주': addr[i][0] = '광주광역시'
    elif addr[i][0] == '대전시': addr[i][0] = '대전광역시'
    elif addr[i][0] == '울산시': addr[i][0] = '울산광역시'
    elif addr[i][0] == '세종시': addr[i][0] = '세종특별자치시'
    elif addr[i][0] == '경기': addr[i][0] = '경기도'
    elif addr[i][0] == '충북': addr[i][0] = '충청북도'
    elif addr[i][0] == '충남': addr[i][0] = '충청남도'
    elif addr[i][0] == '전북': addr[i][0] = '전라북도'
    elif addr[i][0] == '전남': addr[i][0] = '전라남도'
    elif addr[i][0] == '경북': addr[i][0] = '경상북도'
    elif addr[i][0] == '경남': addr[i][0] = '경상남도'
    elif addr[i][0] == '제주': addr[i][0] = '제주특별자치도'
    elif addr[i][0] == '제주도': addr[i][0] = '제주특별자치도'
    elif addr[i][0] == '제주시': addr[i][0] = '제주특별자치도'

In [11]:
addr2.append(' '.join(addr[i]))

In [12]:
addr2

['서울특별시 성북구 고려대로 73']

In [13]:
addr2 = pd.DataFrame(addr2, columns=['address2'])

In [14]:
CB2 = pd.concat([CB, addr2], axis=1)

In [15]:
CB2.head()

,store,address,phone,address2
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 성북구 고려대로 73
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,NaN
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,NaN
3,신사점,서울시 강남구 도산대로 126,02-548-2741,NaN
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,NaN


In [16]:
CB2.to_csv('./CoffeeBean2.csv', encoding='CP949', index=False)

### **5. 분석 모델 구축 및 시각화**

#### **지도 객체 생성하기**
---

포리움 라이브러리를 사용하려면 먼저 folium 패키지를 설치해야 한다.

In [ ]:
!pip install folium

In [18]:
import folium

In [19]:
map_osm = folium.Map(location=[37.559978, 126.975291], zoom_start=16)

In [20]:
map_osm.save('./map.html')

#### **지도에 매장 위치 표시하기**
---